### Exploratory Data Analysis | 8.24.2024

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression


In [1]:
import cudf
#Load the data - Backdoor_Malware.pcap.csv
file_path = '/home/jbenyam/threatdetection/dataset/csv/Backdoor_Malware/Backdoor_Malware.pcap.csv'
df = cudf.read_csv(file_path)


### Initial Data Exploration

In [2]:
# Display basic information about the dataframe
print(df.head()) # first few rows
print(df.info()) # column data types and non-null values



NameError: name 'df' is not defined

### Signature Exploration using Clustering

In [4]:
from cuml.cluster import DBSCAN
import cudf

# Selecting relevant features for clustering
features = df[['HTTP', 'HTTPS', 
               'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 
               'DHCP', 'ARP', 'ICMP', 'IGMP', 'IPv', 'LLC']]

# Normalizing the data
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

#Perform clustering
dbscan = DBSCAN(eps=.5, min_samples=10)
labels = dbscan.fit_predict(features_scaled)

# Add cluster labels to the DataFrame
df['Cluster'] = labels


### Visualize Signatures w. CuxFilter for Interactive Visualization

In [5]:
import plotly.express as px

# Scatter plot
fig = px.scatter(df, x='Header_Length', y='Rate', color='Cluster',
                 title='Scatter plot of Header_Length vs. Rate')
fig.show()


: 

In [ ]:
unique_labels = df['Cluster'].unique()
print(unique_labels)


0    -1
1     0
2     1
3     2
4     9
5     3
6     4
7     5
8     6
9     7
10    8
Name: Cluster, dtype: int32


In [14]:
cdf = cuxfilter.DataFrame(df)
print(cdf.data.head())


   Header_Length  Protocol Type  Time_To_Live        Rate  fin_flag_number  \
0           13.2             17         111.8   21.654112              0.0   
1           11.2             17          63.5  134.621809              0.0   
2           13.6             17          65.6  211.662495              0.0   
3           24.8              6          84.4  155.707333              0.0   
4           10.4             17         118.3  105.440687              0.0   

   syn_flag_number  rst_flag_number  psh_flag_number  ack_flag_number  \
0              0.0              0.0              0.0              0.3   
1              0.1              0.0              0.0              0.0   
2              0.1              0.0              0.0              0.2   
3              0.0              0.0              0.4              0.7   
4              0.0              0.0              0.0              0.1   

   ece_flag_number  ...  Tot sum  Min   Max    AVG         Std  Tot size  \
0              0